In [ ]:
# 문서 임베딩 부터 하자

# 1. 필요한 필드를 추출한다
# 2. LLM이 이해할 수 있도록 자연어로 변환한다.
# 3. 임베딩.

# metadata는 policy id와 policy name만 넣자. 필터링은 RDB에서 하고 있으니까.



In [ ]:
import pandas as pd
df = pd.read_csv('./policy_data.csv')
print(df.head())

In [ ]:
import pandas as pd
import numpy as np

def load_maps_from_excel_final(filepath):
    """
    (최종 버전)
    하나의 엑셀 파일 내의 '코드정보' 시트에서 코드 정보를 읽어와
    분류별 코드-이름 맵 딕셔너리를 생성합니다.
    '코드내용' 컬럼을 사용하며, 오류 발생 시 안전하게 처리합니다.
    
    :param filepath: 엑셀 파일의 전체 경로
    :return: 성공 시 코드맵 딕셔너리, 실패 시 None
    """
    try:
        # 엑셀 파일의 '코드정보' 시트를 읽어옵니다.
        df_codes = pd.read_excel(filepath, sheet_name='코드정보')
        code_maps = {}
        
        # '분류' 컬럼으로 데이터를 그룹화합니다.
        for name, group in df_codes.groupby('분류'):
            # 각 그룹에 '코드'와 '코드내용' 컬럼이 모두 있는지 확인합니다.
            if '코드' in group.columns and '코드내용' in group.columns:
                # '코드' 또는 '코드내용'에 결측치(NaN)가 있는 행은 제외합니다.
                clean_group = group.dropna(subset=['코드', '코드내용'])
                # 최종적으로 코드-이름 딕셔너리를 생성합니다.
                code_maps[name] = dict(zip(clean_group['코드'].astype(str), clean_group['코드내용']))
            else:
                # 필수 컬럼이 없는 그룹은 건너뛰고 사용자에게 경고 메시지를 보여줍니다.
                print(f"⚠️  경고: '{name}' 그룹에 '코드' 또는 '코드내용' 컬럼이 없어 처리에서 제외합니다.")

        print("✅ 코드 정보 파싱 완료!")
        return code_maps
    except FileNotFoundError:
        print(f"🚨 오류: '{filepath}' 파일을 찾을 수 없습니다. 경로를 확인해주세요.")
        return None
    except ValueError as e:
        print(f"🚨 엑셀 파일 내에 '코드정보' 시트가 있는지 확인해주세요. (오류: {e})")
        return None
    except Exception as e:
        print(f"🚨 엑셀 파일 처리 중 예기치 못한 오류 발생: {e}")
        return None

def create_robust_document(row, code_maps):
    """
    (최종 버전)
    파싱된 코드맵과 원본 데이터 행(row)을 바탕으로,
    각종 예외(Null, '제한없음' 등)를 처리하여 의미에 집중한 자연어 설명문을 생성합니다.
    
    :param row: 데이터프레임의 한 행 (pandas.Series)
    :param code_maps: load_maps_from_excel_final 함수로 생성된 코드맵
    :return: 생성된 자연어 설명문 (str)
    """
    # 코드에 해당하는 이름을 찾아주는 내부 함수
    def get_code_name(code_type, code):
        if pd.notna(code):
            # 코드를 문자열로 변환하여 딕셔너리에서 찾습니다.
            return code_maps.get(code_type, {}).get(str(code))
        return None

    parts = []

    # 1. 정책 기본 정보 (이름, 분야, 키워드)
    parts.append(f"정책명은 '{row.get('plcyNm', '정보없음')}'입니다.")
    if pd.notna(row.get('lclsfNm')) and pd.notna(row.get('mclsfNm')):
        parts.append(f"정책 분야는 '{row['lclsfNm']} > {row['mclsfNm']}'입니다.")
    if pd.notna(row.get('plcyKywdNm')):
        parts.append(f"주요 키워드는 '{row['plcyKywdNm'].replace(',', ', ')}'입니다.")

    # 2. 정책 상세 설명 (핵심 내용)
    if pd.notna(row.get('plcyExplnCn')): parts.append(f"정책 설명: {row['plcyExplnCn']}")
    if pd.notna(row.get('plcySprtCn')): parts.append(f"지원 내용: {row['plcySprtCn']}")

    # 3. 자격 요건 (지역/기간 제외)
    condition_parts = []
    # 추가 자격 조건 텍스트가 있으면 추가
    if pd.notna(row.get('addAplyQlfcCndCn')):
        condition_parts.append(f"추가 조건: {row['addAplyQlfcCndCn']}")

    # '제한 없음' 등의 의미를 갖는 키워드 리스트
    unrestricted_keywords = ['관계없음', '제한없음', '학력무관']
    
    # 코드에 해당하는 이름을 찾아, '제한 없음'이 아닐 경우에만 자격 요건에 추가
    mrg_name = get_code_name('mrgSttsCd', row.get('mrgSttsCd'))
    job_name = get_code_name('jobCd', row.get('jobCd'))
    edu_name = get_code_name('schoolCd', row.get('schoolCd'))

    if mrg_name and mrg_name not in unrestricted_keywords: condition_parts.append(f"혼인상태: {mrg_name}")
    if job_name and job_name not in unrestricted_keywords: condition_parts.append(f"취업상태: {job_name}")
    if edu_name and edu_name not in unrestricted_keywords: condition_parts.append(f"학력: {edu_name}")

    if condition_parts:
        parts.append("자격 요건: " + " / ".join(condition_parts))

    # 모든 설명 파트를 하나의 긴 문자열로 결합
    return " ".join(part for part in parts if part and str(part).strip())


if __name__ == '__main__':
    # --- 1. 사용자 설정 영역 ---
    # 👈 여기에 실제 엑셀 파일의 '절대 경로'를 입력해주세요.
    EXCEL_FILE_PATH = './code_table.xlsx'
    
    # 👈 여기에 원본 데이터 CSV 파일의 '절대 경로'를 입력해주세요.
    RAW_DATA_PATH = './policy_data.csv'

    # --- 2. 코드맵 로딩 ---
    code_maps = load_maps_from_excel_final(EXCEL_FILE_PATH)
    
    if code_maps:
        try:
            # --- 3. 원본 데이터 로딩 ---
            df_raw = pd.read_csv(RAW_DATA_PATH)
            print(f"\n✅ 원본 데이터 '{RAW_DATA_PATH}' 로딩 성공!")

            # --- 4. 자연어 설명문 생성 ---
            print("문서 생성을 시작합니다...")
            df_raw['document'] = df_raw.apply(lambda row: create_robust_document(row, code_maps), axis=1)
            print("✅ 최종 자연어 설명문 생성 완료!")
            
            # --- 5. 결과 확인 및 저장 ---
            print("\n--- 생성된 문서 예시 (첫 5개) ---")
            for i, doc in enumerate(df_raw['document'].head()):
                print(f"\n[정책 {i+1}]")
                print(doc)

            # 생성된 문서를 포함하여 최종 결과를 별도의 CSV 파일로 저장
            OUTPUT_PATH = './policies_with_documents.csv'
            df_raw.to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')
            print(f"\n\n✅ 모든 문서가 포함된 결과가 '{OUTPUT_PATH}' 파일로 저장되었습니다.")

        except FileNotFoundError:
            print(f"🚨 오류: 원본 데이터 파일 '{RAW_DATA_PATH}'를 찾을 수 없습니다. 경로를 확인해주세요.")
        except Exception as e:
            print(f"🚨 원본 데이터 처리 중 오류 발생: {e}")
    else:
        print("\n\n🚨 코드맵 로딩 실패로 전체 프로세스를 중단합니다.")

In [ ]:
import pandas as pd
data = pd.read_csv('./policy_data.csv')
data = data[:10]
data.to_csv('./plc_test_data.csv')

In [ ]:
import pandas as pd
import numpy as np

def load_maps_from_excel(filepath):
    try:
        df_codes = pd.read_excel(filepath, sheet_name='코드정보')
        code_maps = {}
        for name, group in df_codes.groupby('분류'):
            if '코드' in group.columns and '코드내용' in group.columns:
                clean_group = group.dropna(subset=['코드', '코드내용'])
                code_maps[name] = dict(zip(clean_group['코드'].astype(str), clean_group['코드내용']))
        print("✅ Excel 파일에서 코드 정보 파싱 완료!")
        return code_maps
    except Exception as e:
        print(f"🚨 Excel 파일 처리 중 오류 발생: {e}")
        return None

def create_final_document(row, code_maps):
    def get_code_name(code_type, code):
        if pd.notna(code) and code_maps:
            first_code = str(code).split(',')[0].strip()
            return code_maps.get(code_type, {}).get(first_code)
        return None

    policy_name = row.get('plcyNm', '이름 정보 없음')
    region = row.get('rgtrInstCdNm', '전국')
    if isinstance(region, str) and ('특별자치도' in region or '광역시' in region or '시' in region):
        region = region.split(' ')[0]

    category = f"{row.get('lclsfNm', '')} > {row.get('mclsfNm', '')}"
    support_content = row.get('plcySprtCn', '지원 내용 정보 없음').strip()
    
    parts = [f"이 정책은 '{region}'에서 시행하는 '{policy_name}'입니다."]
    parts.append(f"정책 분야는 '{category}'이며, '{support_content}'을 지원합니다.")

    conditions = []
    
    min_age, max_age = row.get('sprtTrgtMinAge'), row.get('sprtTrgtMaxAge')
    if pd.notna(max_age) and max_age > 0:
        if pd.notna(min_age) and min_age > 0:
            conditions.append(f"만 {int(min_age)}세에서 {int(max_age)}세 사이의 청년")
        else:
            conditions.append(f"만 {int(max_age)}세 이하의 청년")

    if pd.notna(row.get('earnEtcCn')) and str(row.get('earnEtcCn')).strip():
        conditions.append(f"소득 조건은 '{row['earnEtcCn']}'을 따릅니다.")
    
    # --- 수정된 자격 조건 로직 ---
    unrestricted_keywords = ['관계없음', '제한없음', '학력무관', '무관']
    
    # 결혼 상태
    mrg_name = get_code_name('mrgSttsCd', row.get('mrgSttsCd'))
    if mrg_name and mrg_name not in unrestricted_keywords:
        conditions.append(f"혼인 상태는 '{mrg_name}'이어야 합니다.")

    # 취업 상태
    job_name = get_code_name('jobCd', row.get('jobCd'))
    if job_name:
        if job_name in unrestricted_keywords:
             conditions.append("취업 상태와 관계없이 지원 가능합니다.")
        else:
             conditions.append(f"취업 상태는 '{job_name}'이어야 합니다.")
    
    # 학력
    edu_name = get_code_name('schoolCd', row.get('schoolCd'))
    if edu_name:
        if edu_name in unrestricted_keywords:
            conditions.append("학력과 관계없이 지원 가능합니다.")
        else:
            conditions.append(f"학력 조건은 '{edu_name}'입니다.")
    # --- 로직 수정 끝 ---

    if pd.notna(row.get('addAplyQlfcCndCn')) and str(row.get('addAplyQlfcCndCn')).strip():
        conditions.append(f"추가 자격: {row['addAplyQlfcCndCn']}")

    if conditions:
        parts.append("지원 대상은 " + ", ".join(filter(None, conditions)) + "입니다.")
    
    if pd.notna(row.get('plcyAplyMthdCn')) and str(row.get('plcyAplyMthdCn')).strip():
        parts.append(f"신청 방법은 {row['plcyAplyMthdCn']}입니다.")

    return " ".join(parts)


if __name__ == '__main__':
    POLICY_CSV_PATH = './policy_data.csv'
    CODE_EXCEL_PATH = './code_table.xlsx'
    OUTPUT_CSV_PATH = './policies_with_documents_final.csv'

    try:
        df_raw = pd.read_csv(POLICY_CSV_PATH, encoding='utf-8')
        print(f"✅ 원본 CSV 데이터 '{POLICY_CSV_PATH}' 로딩 성공!")
    except FileNotFoundError:
        print(f"🚨 오류: '{POLICY_CSV_PATH}' 파일을 찾을 수 없습니다.")
        exit()

    df_raw['sprtTrgtMinAge'] = pd.to_numeric(df_raw['sprtTrgtMinAge'], errors='coerce')
    df_raw['sprtTrgtMaxAge'] = pd.to_numeric(df_raw['sprtTrgtMaxAge'], errors='coerce')

    code_maps = load_maps_from_excel(CODE_EXCEL_PATH)

    if code_maps:
        print("\n문서 생성을 시작합니다...")
        df_raw['document'] = df_raw.apply(lambda row: create_final_document(row, code_maps), axis=1)
        print("✅ 최종 자연어 설명문 생성 완료!")
        df_raw.to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
        print(f"\n✅ 모든 문서가 포함된 최종 결과가 '{OUTPUT_CSV_PATH}' 파일로 저장되었습니다.")
    else:
        print("\n\n🚨 코드맵 로딩 실패로 전체 프로세스를 중단합니다.")